In [5]:
import random
import time
import matplotlib.pyplot as plt
from scipy import stats
from IPython.display import clear_output

import fastlob as lob

In [6]:
def generate_orders(T: int, midprice: float):
    result = list()
    
    for _ in range(T):
    
        n_ask_limits = stats.poisson.rvs(500)
        n_bid_limits = stats.poisson.rvs(500)
    
        ask_limits_price = stats.expon.rvs(loc=midprice, scale=1, size=n_ask_limits)
        bid_limits_price = -stats.expon.rvs(loc=midprice, scale=1, size=n_bid_limits) + 2*midprice
    
        ask_limits_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_ask_limits)
        bid_limits_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_bid_limits)
    
        ask_limits_params = [lob.OrderParams(lob.OrderSide.ASK, p, q) for (p, q) in zip(ask_limits_price, ask_limits_quantities)]
        bid_limits_params = [lob.OrderParams(lob.OrderSide.BID, p, q) for (p, q) in zip(bid_limits_price, bid_limits_quantities)]
    
        n_markets = stats.poisson.rvs(100)
    
        markets_price = stats.norm.rvs(loc=midprice, scale=2, size=n_markets)
        markets_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_markets)
        markets_bid_or_ask = [random.choice((lob.OrderSide.BID, lob.OrderSide.ASK)) for _ in range(n_markets)]
    
        markets_params = [lob.OrderParams(s, p, q) for (s, p, q) in zip(markets_bid_or_ask, markets_price, markets_quantities)]
    
        orders = ask_limits_params + bid_limits_params + markets_params
        random.shuffle(orders)
        
        result.append(orders)
        
    return result

In [7]:
def simulate(orders):
    ob = lob.Orderbook('Simulation')
    ob.start()

    for o in orders:
        ob.process_many(o)
        ob.render()
        time.sleep(0.1)
        clear_output(wait=True)
        
    ob.stop()

In [8]:
orders = generate_orders(100, 657.98)
simulate(orders)

   [ORDER-BOOK Simulation]

   ...(678 more asks)
 - 658.09 | 460 | 22993.75 | 15131956.9375
 - 658.08 | 469 | 23227.03 | 15285243.9024
 - 658.07 | 488 | 24907.82 | 16391089.1074
 - 658.06 | 490 | 24404.62 | 16059704.2372
 - 658.05 | 495 | 25068.83 | 16496543.5815
 - 658.04 | 499 | 24691.68 | 16248113.1072
 - 658.03 | 172 | 08773.26 | 5773068.2778
 - 658.02 | 007 | 00335.55 | 220798.6110
 - 658.01 | 001 | 00019.67 | 12943.0567
 - 657.99 | 001 | 00001.28 | 842.2272
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 - 657.95 | 007 | 00439.65 | 289267.7175
 - 657.94 | 009 | 00521.29 | 342977.5426
 - 657.93 | 208 | 09690.14 | 6375433.8102
 - 657.92 | 455 | 22840.25 | 15027057.2800
 - 657.91 | 470 | 23681.66 | 15580400.9306
 - 657.90 | 483 | 24085.67 | 15845962.2930
 - 657.89 | 470 | 22964.04 | 15107812.2756
 - 657.88 | 462 | 24295.74 | 15983681.4312
 - 657.87 | 438 | 21253.60 | 13982105.8320
 - 657.86 | 448 | 22266.60 | 14648305.4760
   ...(676 more bids)

    Spread = 0.04, Mid-price = 657.970
